# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import cleaned data from previous activity
cities_df = pd.read_csv('city_data.csv')

#Delete extra column
del cities_df['Unnamed: 0']

cities_df.head()

,city,country,lat,lng,humidity,temp,max_temp,clouds,wind
0,georgetown,MY,5.4112,100.3354,88,75.97,77.00,20,5.61
1,cape town,ZA,-33.9258,18.4232,82,67.10,69.01,0,16.11
2,kapaa,US,22.0752,-159.3190,69,78.35,78.80,20,18.41
3,nelson bay,AU,-32.7167,152.1500,78,72.61,73.99,75,9.22
4,east london,ZA,-33.0153,27.9116,79,70.16,70.16,92,11.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '700px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

#Create heat layer
heat_layer = gmaps.heatmap_layer(cities_df[['lat', 'lng']], weights= cities_df['humidity'], 
                                dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

#### My Ideal Vacation Weather:
* Humidity 25-55 %
* Temperature 60-80 (F)
* Wind Speed 0-5mph
* Cloudiness 0-50% cover

In [4]:
#Use loc statment to filter to the above parameters
myideal_df = cities_df.loc[(cities_df['humidity'] >= 25) & (cities_df['humidity'] < 55) & (cities_df['temp'] > 60) & (cities_df['temp'] <= 80) & (cities_df['clouds'] <= 50) & (cities_df['wind'] <= 5)]

#Narrow down to only 5
myideal_df = myideal_df.head()

myideal_df

,city,country,lat,lng,humidity,temp,max_temp,clouds,wind
14,nuevo progreso,MX,23.1667,-97.9500,51,68.00,68.00,44,4.00
93,san rafael,AR,-34.6177,-68.3301,37,69.58,69.58,7,4.85
124,coihaique,CL,-45.5752,-72.0662,43,75.20,75.20,0,3.44
183,vallenar,CL,-28.5708,-70.7581,50,66.43,66.43,47,2.89
246,deniliquin,AU,-35.5333,144.9667,51,68.00,68.00,0,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Narrow down to the data needed for the places api request
hotel_df = myideal_df[['city', 'country', 'lat', 'lng']]

#add in a row for hotel
hotel_df['Hotel Name'] = ''

hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,city,country,lat,lng,Hotel Name
14,nuevo progreso,MX,23.1667,-97.9500,
93,san rafael,AR,-34.6177,-68.3301,
124,coihaique,CL,-45.5752,-72.0662,
183,vallenar,CL,-28.5708,-70.7581,
246,deniliquin,AU,-35.5333,144.9667,


In [6]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

for index, row in hotel_df.iterrows():
    params = {
        'key': g_key,
        'location': f"{row['lat']},{row['lng']}",
        'radius': '5000',
        'type': 'lodging'
    }

    hotel_data = requests.get(base_url, params=params).json()

    #print(json.dumps(hotel_data, indent=4, sort_keys=True))
    try:
        print(f"Retriving data for {row['city']}...")
        hotel_df.loc[index, 'Hotel Name'] = hotel_data['results'][0]['name']
    except:
        print(f"No hotels in radius for {row['city']}...")
        
hotel_df

Retriving data for nuevo progreso...
No hotels in radius for nuevo progreso...
Retriving data for san rafael...
No hotels in radius for san rafael...
Retriving data for coihaique...
No hotels in radius for coihaique...
Retriving data for vallenar...
No hotels in radius for vallenar...
Retriving data for deniliquin...
No hotels in radius for deniliquin...


,city,country,lat,lng,Hotel Name
14,nuevo progreso,MX,23.1667,-97.9500,
93,san rafael,AR,-34.6177,-68.3301,
124,coihaique,CL,-45.5752,-72.0662,
183,vallenar,CL,-28.5708,-70.7581,
246,deniliquin,AU,-35.5333,144.9667,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…